You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

**Content Plan on Artificial Intelligence**

**Introduction:**
Artificial Intelligence (AI) has been making significant advancements in various industries, revolutionizing the way we live and work. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of AI, providing valuable insights for our target audience.

**Target Audience:**
Our target audience consists of tech enthusiasts, business professionals, and individuals curious about the potential of AI. They 

I now can give a great answer

Final Answer: 

# The Evolution of Artificial Intelligence: Trends, Players, and Ethics

**Introduction to Artificial Intelligence**

Artificial Intelligence (AI) has become a driving force in reshaping industries and transforming the way we interact with technology. From self-driving cars to personalized recommendations in e-commerce, AI is revolutionizing the world around us. In this blog post, we will delve into the latest trends, key players, and ethical implications of AI, providing valuable insights for tech enthusiasts, business professionals, and individuals curious about the potential of this rapidly advancing technology.

**Latest Trends in AI**

Machine Learning, Natural Language Processing, and Computer Vision are at the forefront of AI innovation. Machine Learning algorithms enable computers to learn and improve from experience, while Natural Language Processing allows machines to understand and generate human language. Computer Vision, on th

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence: Trends, Players, and Ethics

**Introduction to Artificial Intelligence**

Artificial Intelligence (AI) has become a driving force in reshaping industries and transforming the way we interact with technology. From self-driving cars to personalized recommendations in e-commerce, AI is revolutionizing the world around us. In this blog post, we will delve into the latest trends, key players, and ethical implications of AI, providing valuable insights for tech enthusiasts, business professionals, and individuals curious about the potential of this rapidly advancing technology.

**Latest Trends in AI**

Machine Learning, Natural Language Processing, and Computer Vision are at the forefront of AI innovation. Machine Learning algorithms enable computers to learn and improve from experience, while Natural Language Processing allows machines to understand and generate human language. Computer Vision, on the other hand, focuses on enabling machines to interpret and understand visual information. These trends are driving the development of cutting-edge AI applications that are reshaping industries and improving efficiency.

**Key Players in the AI Industry**

Google, Amazon, and IBM are among the key players in the AI industry, leading the way in research and innovation. Google's deep learning algorithms power many of its products and services, while Amazon's AI-driven recommendations have transformed the e-commerce landscape. IBM's Watson AI platform is revolutionizing healthcare, finance, and other industries with its cognitive computing capabilities. These companies are shaping the future of AI and setting the standard for innovation in the industry.

**Noteworthy News in Artificial Intelligence**

Recent breakthroughs in AI research, such as OpenAI's GPT-3 language model, have garnered widespread attention for their potential to revolutionize various fields. AI applications in healthcare are improving patient outcomes and driving medical advancements, while AI in self-driving cars is paving the way for a future of autonomous transportation. These noteworthy developments highlight the vast potential of AI to transform industries and improve the quality of life for individuals around the world.

**Impact of AI on Industries**

AI is driving automation in manufacturing, enabling businesses to streamline processes and increase efficiency. Personalized recommendations in e-commerce are enhancing the customer experience and driving sales, while AI in customer service is revolutionizing how businesses interact with consumers. These applications of AI are reshaping industries and creating new opportunities for growth and innovation.

**Ethical Implications of Artificial Intelligence**

Despite its numerous benefits, AI also poses ethical challenges, such as bias in algorithms, privacy concerns, and the fear of job displacement. Ensuring that AI systems are fair and unbiased is crucial to building trust and safeguarding against discrimination. Protecting privacy and addressing job displacement are also key considerations in the ethical development and deployment of AI technologies.

**Conclusion and Call to Action**

In conclusion, Artificial Intelligence is a powerful and transformative technology that is reshaping industries and driving innovation. By staying informed about the latest trends, key players, and ethical implications of AI, we can better understand its potential impact on our lives and industries. As we navigate the evolving landscape of AI, it is essential to prioritize ethical considerations and work towards creating a future where AI benefits society as a whole.

As we continue to explore the possibilities of AI, let us strive to harness its potential for good and address the ethical challenges that come with its advancement. Together, we can shape a future where Artificial Intelligence enhances our lives and creates a more equitable and sustainable world for all.

---
This blog post aims to provide a comprehensive overview of Artificial Intelligence, addressing the latest trends, key players, and noteworthy news in the industry. By incorporating SEO keywords and relevant data sources, we have created engaging and informative content for our target audience.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "fredagsmys ritual"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on fredagsmys ritual.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

# The Ultimate Guide to Fredagsmys Ritual: How to Enjoy a Cozy Friday Night

## Introduction

Fredagsmys ritual, originating from Sweden, is a beloved tradition that involves creating a cozy and relaxed atmosphere to enjoy a Friday night. This ritual is all about unwinding, connecting with loved ones, and indulging in comfort food and activities. The concept of fredagsmys has gained popularity beyond Sweden, with people around the world embracing the idea of a cozy Friday night in.

## Latest Trends in Fredagsmys R

In [14]:
Markdown(result)

# The Ultimate Guide to Fredagsmys Ritual: How to Enjoy a Cozy Friday Night

## Introduction

The Fredagsmys ritual, originating from Sweden, is a cherished tradition that revolves around creating a cozy and laid-back atmosphere to savor a Friday night. It is all about unwinding, connecting with loved ones, and indulging in comfort food and activities. The concept of fredagsmys has transcended its Swedish roots and garnered global popularity, with individuals worldwide embracing the idea of a relaxed Friday night in.

## Latest Trends in Fredagsmys Ritual

In today's fast-paced world, many individuals are leveraging technology to elevate their fredagsmys experience. Virtual fredagsmys gatherings have witnessed a surge in popularity, enabling friends and family members to connect from different locations. Moreover, do-it-yourself concepts for crafting a cozy ambiance at home have gained traction, with people exploring innovative decorations, lighting arrangements, and themed activities to make their Friday nights truly special.

## Key Players in Fredagsmys Ritual

Numerous influencers and brands have played a pivotal role in championing the fredagsmys culture. Influencers on various social media platforms share their fredagsmys routines, providing inspiration and suggestions for cultivating a cozy atmosphere. Brands have also seized the opportunity by offering products like snug blankets, scented candles, and comfort food items that enrich the fredagsmys experience.

## Noteworthy News in Fredagsmys Ritual

Recent initiatives aimed at promoting fredagsmys have surfaced, including events that celebrate the tradition and underscore its advantages. Studies have highlighted that engaging in fredagsmys activities can have a positive impact on mental health and overall well-being, underscoring the significance of relaxation and downtime in today's bustling world.

## Target Audience Analysis

Millennials and Gen Z individuals stand out as key demographics seeking relaxation and connection through the fredagsmys ritual. For these generations, fredagsmys provides a respite from the digital realm and an opportunity to unwind with loved ones. Families also recognize the value of the ritual, utilizing it as a chance for bonding and quality time together.

## Conclusion

In conclusion, the fredagsmys ritual stands as a delightful tradition that champions relaxation, connection, and well-being. By incorporating the latest trends, key players, and noteworthy news in the fredagsmys culture, individuals can elevate their Friday night experience and prioritize self-care. I urge readers to consider integrating fredagsmys into their own routines and sharing their experiences on social media using the hashtag #fredagsmys. Let's embrace the cozy Friday night tradition and relish the benefits it brings to our lives.

*Sources: Research studies on the benefits of relaxation and mindfulness, interviews with experts on the importance of self-care and downtime*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).